<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/Copie2_M2_ATAL_2021_22_02_NER_with_BiLSTM_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Recent Advances in Sequence Labeling from Deep Learning Models

Les approches pour l'étiquetage de séquence fondées sur les réseaux de neurones profonds compte trois étapes :
1. The embedding module is the first stage that maps words into their distributed representations (pretrained word embeddings, character-
level representations, hand-crafted features and sentence-level
representations). 
2. The context encoder module extracts contextual features (e.g. RNN/Bi-LSTM, CNN)
3. and the inference module predict labels and generate optimal label sequence as output of the model (e.g. SoftMax, CRF, RNN). 

[Zhiyong He, Zanbo Wang, Sheng Jiang. A Survey on Recent Advances in Sequence Labeling from Deep Learning Models. Published 13 November 2020. Computer Science. ArXiv](https://arxiv.org/pdf/2011.06727.pdf)


---
# Bref historique des systèmes de NER neuronaux

(on ne vous demande pas de lire les articles)

* L'architecture "SENNA", novatrice dans l'idée de la résolution des tâches du TAL avec un modèle de langue neuronal (incluant notamment une méthode de construction de "pretrained word embeddings") : R. Collobert, J. Weston, L. Bottou, M. Karlen, K. Kavukcuoglu and P. Kuksa. Natural Language Processing (Almost) from Scratch, Journal of Machine Learning Research (JMLR), 2011. ; [[article]](http://ronan.collobert.com/pub/matos/2011_nlp_jmlr.pdf) ; [[implémentation]](https://ronan.collobert.com/senna/)
* Premier article à appliquer les BiLSTM-CRF au NER : Zhiheng Huang, Wei Xu, Kai Yu, Bidirectional LSTM-CRF Models for Sequence Tagging, Arxiv, Computation and Language, Submitted on 9 Aug 2015 ; [[article]](https://arxiv.org/pdf/1508.01991.pdf) ; [[implémentation1]](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html) (tutoriel avancé de pytorch) ; [[implémentation2]](https://github.com/ZubinGou/NER-BiLSTM-CRF-PyTorch) (inclut aussi un modèle Bi-LSTM-CNN-CRF) ; [[implémentation3]](https://github.com/jidasheng/bi-lstm-crf)  ; [[implémentation4]](http://www.gabormelli.com/RKB/index.php?title=Bidirectional_LSTM/CRF_(BiLTSM-CRF)_Training_System) ; [[implémentation5]](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html) (avec tensorflow)
* BiLSTM-CNN-CRF Implementation for Sequence Tagging (extension with the ELMo representations) : Reimers, Nils, and Gurevych, Iryna, Reporting Score Distributions Makes a Difference: Performance Study of LSTM-networks for Sequence Tagging, Proceedings of the 2017 Conference on Empirical Methods in Natural Language Processing (EMNLP), September 2017, Copenhagen, Denmark, 338-348 ; [[article]](http://aclweb.org/anthology/D17-1035) ; [[implémentation]](https://github.com/UKPLab/emnlp2017-bilstm-cnn-crf)
* Le 3e modèle le plus performant en 2020 sur la tâche NER sans ressources externes : Ying Luo, Fengshun Xiao, and Hai Zhao. Hierarchical contextualized representation for named entity recognition. In AAAI, pages 8441–8448, 2020 ; [[implémentation]](https://github.com/cslydia/Hire-NER) ; Utilise [NCRF++: An Open-source Neural Sequence Labeling Toolkit](https://github.com/jiesutd/NCRFpp)


---
# Bidirectional LSTM-CRF Implémentation de Huang et al. 2015 

Le code dans les cellules suivantes provient de l'[implémentation 2](https://github.com/jidasheng/bi-lstm-crf/) du système de (Huang et al., 2015). Celle-ci s'appuie sur la bibliothèque pytorch.



### VOTRE TRAVAIL 
* Exécutez les cellules sans passer trop de temps à comprendre les détails de l'implémentation. Répondez aux questions quand vous y êtes invité.


## Installation des dépendances 

(éventuellement exécuter plusieurs fois la cellule pour faire disparaître les erreurs)

In [ ]:
# https://stackoverflow.com/questions/54358280/packed-padded-sequence-gives-error-when-used-with-gpu
!pip install torch==1.6.0 torchvision==0.7.0

Test pour déterminer si le hardware de votre machine dispose d'un gpu. Dans un premier temps, sous gcolab, garder le type d'exécution natif "cpu" afin d'avoir une idée des temps d'entraînement de l'architecture.

In [ ]:
import torch
# Get cpu or gpu device for training.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## Définition d'une cellule neuronale CRF 

* Trouve la séquence d'étiquettes la plus probable correspondant à une séquence de mots donnée
* Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/model/crf.py
* Pour en savoir plus sur les CRF, vous pouvez lire : http://www.cs.columbia.edu/~mcollins/crf.pdf.



In [ ]:
import torch
import torch.nn as nn

def log_sum_exp(x):
    """calculate log(sum(exp(x))) = max(x) + log(sum(exp(x - max(x))))
    """
    max_score = x.max(-1)[0]
    return max_score + (x - max_score.unsqueeze(-1)).exp().sum(-1).log()


IMPOSSIBLE = -1e4

class CRF(nn.Module):
    """General CRF module.
    The CRF module contain a inner Linear Layer which transform the input from features space to tag space.

    :param in_features: number of features for the input
    :param num_tag: number of tags. DO NOT include START, STOP tags, they are included internal.
    """

    def __init__(self, in_features, num_tags):
        super(CRF, self).__init__()

        self.num_tags = num_tags + 2
        self.start_idx = self.num_tags - 2
        self.stop_idx = self.num_tags - 1

        self.fc = nn.Linear(in_features, self.num_tags)

        # transition factor, Tij mean transition from j to i
        self.transitions = nn.Parameter(torch.randn(self.num_tags, self.num_tags), requires_grad=True)
        self.transitions.data[self.start_idx, :] = IMPOSSIBLE
        self.transitions.data[:, self.stop_idx] = IMPOSSIBLE

    def forward(self, features, masks):
        """decode tags

        :param features: [B, L, C], batch of unary scores
        :param masks: [B, L] masks
        :return: (best_score, best_paths)
            best_score: [B]
            best_paths: [B, L]
        """
        features = self.fc(features)
        return self.__viterbi_decode(features, masks[:, :features.size(1)].float())

    def loss(self, features, ys, masks):
        """negative log likelihood loss
        B: batch size, L: sequence length, D: dimension

        :param features: [B, L, D]
        :param ys: tags, [B, L]
        :param masks: masks for padding, [B, L]
        :return: loss
        """
        features = self.fc(features)

        L = features.size(1)
        masks_ = masks[:, :L].float()

        forward_score = self.__forward_algorithm(features, masks_)
        gold_score = self.__score_sentence(features, ys[:, :L].long(), masks_)
        loss = (forward_score - gold_score).mean()
        return loss

    def __score_sentence(self, features, tags, masks):
        """Gives the score of a provided tag sequence

        :param features: [B, L, C]
        :param tags: [B, L]
        :param masks: [B, L]
        :return: [B] score in the log space
        """
        B, L, C = features.shape

        # emission score
        emit_scores = features.gather(dim=2, index=tags.unsqueeze(-1)).squeeze(-1)

        # transition score
        start_tag = torch.full((B, 1), self.start_idx, dtype=torch.long, device=tags.device)
        tags = torch.cat([start_tag, tags], dim=1)  # [B, L+1]
        trans_scores = self.transitions[tags[:, 1:], tags[:, :-1]]

        # last transition score to STOP tag
        last_tag = tags.gather(dim=1, index=masks.sum(1).long().unsqueeze(1)).squeeze(1)  # [B]
        last_score = self.transitions[self.stop_idx, last_tag]

        score = ((trans_scores + emit_scores) * masks).sum(1) + last_score
        return score

    def __viterbi_decode(self, features, masks):
        """decode to tags using viterbi algorithm

        :param features: [B, L, C], batch of unary scores
        :param masks: [B, L] masks
        :return: (best_score, best_paths)
            best_score: [B]
            best_paths: [B, L]
        """
        B, L, C = features.shape

        bps = torch.zeros(B, L, C, dtype=torch.long, device=features.device)  # back pointers

        # Initialize the viterbi variables in log space
        max_score = torch.full((B, C), IMPOSSIBLE, device=features.device)  # [B, C]
        max_score[:, self.start_idx] = 0

        for t in range(L):
            mask_t = masks[:, t].unsqueeze(1)  # [B, 1]
            emit_score_t = features[:, t]  # [B, C]

            # [B, 1, C] + [C, C]
            acc_score_t = max_score.unsqueeze(1) + self.transitions  # [B, C, C]
            acc_score_t, bps[:, t, :] = acc_score_t.max(dim=-1)
            acc_score_t += emit_score_t
            max_score = acc_score_t * mask_t + max_score * (1 - mask_t)  # max_score or acc_score_t

        # Transition to STOP_TAG
        max_score += self.transitions[self.stop_idx]
        best_score, best_tag = max_score.max(dim=-1)

        # Follow the back pointers to decode the best path.
        best_paths = []
        bps = bps.cpu().numpy()
        for b in range(B):
            best_tag_b = best_tag[b].item()
            seq_len = int(masks[b, :].sum().item())

            best_path = [best_tag_b]
            for bps_t in reversed(bps[b, :seq_len]):
                best_tag_b = bps_t[best_tag_b]
                best_path.append(best_tag_b)
            # drop the last tag and reverse the left
            best_paths.append(best_path[-2::-1])

        return best_score, best_paths

    def __forward_algorithm(self, features, masks):
        """calculate the partition function with forward algorithm.
        TRICK: log_sum_exp([x1, x2, x3, x4, ...]) = log_sum_exp([log_sum_exp([x1, x2]), log_sum_exp([x3, x4]), ...])

        :param features: features. [B, L, C]
        :param masks: [B, L] masks
        :return:    [B], score in the log space
        """
        B, L, C = features.shape

        scores = torch.full((B, C), IMPOSSIBLE, device=features.device)  # [B, C]
        scores[:, self.start_idx] = 0.
        trans = self.transitions.unsqueeze(0)  # [1, C, C]

        # Iterate through the sentence
        for t in range(L):
            emit_score_t = features[:, t].unsqueeze(2)  # [B, C, 1]
            score_t = scores.unsqueeze(1) + trans + emit_score_t  # [B, 1, C] + [1, C, C] + [B, C, 1] => [B, C, C]
            score_t = log_sum_exp(score_t)  # [B, C]

            mask_t = masks[:, t].unsqueeze(1)  # [B, 1]
            scores = score_t * mask_t + scores * (1 - mask_t)
        scores = log_sum_exp(scores + self.transitions[self.stop_idx])
        return scores


### VOTRE TRAVAIL

Dans GColab, Faire Outils > Paramètres > Cocher "affichage de la numérotation des lignes"

* Quel est le nom de la _loss function_ ? A quelle ligne est-ce spécifiée ?
* En quelques mots, à quoi sert l'algorithme de Viterbi ? Cherchez sur le web...

Eventuellement, en apprendre davantage sur quelques [_loss functions_](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/).

## Définition d'une architecture neuronale Bi-LSTM CRF

La classe suivante implémente un modèle Bi-LSTM CRF
- Construction des embeddings de la phrase
- Capture du contexte avec une cellule RNN 
- Prédiction de la séquence d'étiquetage à l'aide de la cellule CRF qui prend comme input la sortie du RNN

Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/model/model.py


In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class BiRnnCrf(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_rnn_layers=1, rnn="lstm"):
        super(BiRnnCrf, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tagset_size = tagset_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        RNN = nn.LSTM if rnn == "lstm" else nn.GRU
        self.rnn = RNN(embedding_dim, hidden_dim // 2, num_layers=num_rnn_layers,
                       bidirectional=True, batch_first=True)
        self.crf = CRF(hidden_dim, self.tagset_size)

    def __build_features(self, sentences):
        masks = sentences.gt(0)
        embeds = self.embedding(sentences.long())

        seq_length = masks.sum(1)
        sorted_seq_length, perm_idx = seq_length.sort(descending=True)
        embeds = embeds[perm_idx, :]

        pack_sequence = pack_padded_sequence(embeds, lengths=sorted_seq_length,  batch_first=True)
        packed_output, _ = self.rnn(pack_sequence)
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)
        _, unperm_idx = perm_idx.sort()
        lstm_out = lstm_out[unperm_idx, :]

        return lstm_out, masks

    def loss(self, xs, tags):
        features, masks = self.__build_features(xs)
        loss = self.crf.loss(features, tags, masks=masks)
        return loss

    def forward(self, xs):
        # Get the emission scores from the BiLSTM
        features, masks = self.__build_features(xs)
        scores, tag_seq = self.crf(features, masks)
        return scores, tag_seq

### VOTRE TRAVAIL

Dans GColab, Faire Outils > Paramètres > Cocher "affichage de la numérotation des lignes"

* L'implémentation offre deux types de cellules RNN possibles. Lesquels ? Indiquez le numéro de ligne où ce choix est spécifié.
* Après la représentation en embeddings des phrase et avant le passage à la cellule RNN, quel type de traitement est réalisé ? Indiquez le numéro de ligne où ce traitement est spécifié. 

---
## Définition des prétraitement des données 


D'abord la définition de méthodes "utils" pour la phase de prétraitement à savoir la sauvegarde et le chargement de fichiers de configuration e.g. vocabulaire, jeu d'étiquettes, paramètres du modèle neuronal (dimension des embeddings...), partition des données générées...

Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/app/preprocessing/utils.py

In [ ]:
import json

START_TAG = "<START>"
STOP_TAG = "<STOP>"

PAD = "<PAD>"
OOV = "<OOV>"


def save_json_file(obj, file_path):
    with open(file_path, "w", encoding="utf8") as f:
        f.write(json.dumps(obj, ensure_ascii=False))


def load_json_file(file_path):
    with open(file_path, encoding="utf8") as f:
        return json.load(f)

Puis la classe de pré-traitement des données qui sera initialisé à l'aide des chemins des fichiers contenant le vocabulaire, le jeu d'étiquettes et les données annotées (phrases découpées en mots avec étiquettes). Outre charger ces fichiers de configuration et données, la classe partitionne les données en ensemble d'entrainement, de validation et de tests (d'après les paramètres spécifiés par défaut ou à l'appel du système). Les données sont aussi "vectorisées". Il s'agit essentiellement d'une substitution des mots des phrases par leur identifiant numérique correspondant à une entrée dans le vocabulaire donné.

https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/app/preprocessing/preprocess.py

In [ ]:
from os.path import join, exists
import numpy as np
from tqdm import tqdm
import torch

#FILE_VOCAB = "vocab.json"
#FILE_TAGS = "tags.json"
#FILE_DATASET = "dataset.txt"
#FILE_DATASET_CACHE = "dataset_cache_{}.npz"


class Preprocessor:
    def __init__(self, config_dir, save_config_dir=None, verbose=True):
        self.config_dir = config_dir
        self.verbose = verbose

        self.vocab, self.vocab_dict = self.__load_list_file(FILE_VOCAB, offset=1, verbose=verbose)
        self.tags, self.tags_dict = self.__load_list_file(FILE_TAGS, verbose=verbose)
        if save_config_dir:
            self.__save_config(save_config_dir)

        self.PAD_IDX = 0
        self.OOV_IDX = len(self.vocab)
        self.__adjust_vocab()

    def __load_list_file(self, file_name, offset=0, verbose=False):
        file_path = join(self.config_dir, file_name)
        if not exists(file_path):
            raise ValueError('"{}" file does not exist.'.format(file_path))
        else:
            elements = load_json_file(file_path)
            elements_dict = {w: idx + offset for idx, w in enumerate(elements)}
            if verbose:
                print("config {} loaded".format(file_path))
            return elements, elements_dict

    def __adjust_vocab(self):
        self.vocab.insert(0, PAD)
        self.vocab_dict[PAD] = 0

        self.vocab.append(OOV)
        self.vocab_dict[OOV] = len(self.vocab) - 1

    def __save_config(self, dst_dir):
        char_file = join(dst_dir, FILE_VOCAB)
        save_json_file(self.vocab, char_file)

        tag_file = join(dst_dir, FILE_TAGS)
        save_json_file(self.tags, tag_file)

        if self.verbose:
            print("tag dict file => {}".format(tag_file))
            print("tag dict file => {}".format(char_file))

    @staticmethod
    def __cache_file_path(corpus_dir, max_seq_len):
        return join(corpus_dir, FILE_DATASET_CACHE.format(max_seq_len))

    def load_dataset(self, corpus_dir, val_split, test_split, max_seq_len):
        """load the train set
        :return: (xs, ys)
            xs: [B, L]
            ys: [B, L, C]
        """
        ds_path = self.__cache_file_path(corpus_dir, max_seq_len)
        if not exists(ds_path):
            xs, ys = self.__build_corpus(corpus_dir, max_seq_len)
        else:
            print("loading dataset {} ...".format(ds_path))
            dataset = np.load(ds_path)
            xs, ys = dataset["xs"], dataset["ys"]

        xs, ys = map(
            torch.tensor, (xs, ys)
        )

        # split the dataset
        total_count = len(xs)
        assert total_count == len(ys)
        val_count = int(total_count * val_split)
        test_count = int(total_count * test_split)
        train_count = total_count - val_count - test_count
        assert train_count > 0 and val_count > 0

        indices = np.cumsum([0, train_count, val_count, test_count])
        datasets = [(xs[s:e], ys[s:e]) for s, e in zip(indices[:-1], indices[1:])]
        print("datasets loaded:")
        for (xs_, ys_), name in zip(datasets, ["train", "val", "test"]):
            print("\t{}: {}, {}".format(name, xs_.shape, ys_.shape))
        return datasets

    def decode_tags(self, batch_tags):
        batch_tags = [
            [self.tags[t] for t in tags]
            for tags in batch_tags
        ]
        return batch_tags

    def sent_to_vector(self, sentence, max_seq_len=0):
        max_seq_len = max_seq_len if max_seq_len > 0 else len(sentence)
        vec = [self.vocab_dict.get(c, self.OOV_IDX) for c in sentence[:max_seq_len]]
        return vec + [self.PAD_IDX] * (max_seq_len - len(vec))

    def tags_to_vector(self, tags, max_seq_len=0):
        max_seq_len = max_seq_len if max_seq_len > 0 else len(tags)
        vec = [self.tags_dict[c] for c in tags[:max_seq_len]]
        return vec + [0] * (max_seq_len - len(vec))

    def __build_corpus(self, corpus_dir, max_seq_len):
      # remove cache files !!!
        file_path = join(corpus_dir, FILE_DATASET)
        xs, ys = [], []
        with open(file_path, encoding="utf8") as f:
            for idx, line in tqdm(enumerate(f), desc="parsing {}".format(file_path)):
                fields = line.strip().split("\t")
                if len(fields) != 2:
                    raise ValueError("format error in line {}, tabs count: {}".format(idx + 1, len(fields) - 1))

                sentence, tags = fields


                try:
                    if sentence[0] == "[":
                        sentence = json.loads(sentence)
                    tags = json.loads(tags)

                    #print ('Debug: sentence', sentence)
                    #print ('Debug: tags', tags)

                    xs.append(self.sent_to_vector(sentence, max_seq_len=max_seq_len))
                    ys.append(self.tags_to_vector(tags, max_seq_len=max_seq_len))
                    if len(sentence) != len(tags):
                        raise ValueError('"sentence length({})" != "tags length({})" in line {}"'.format(
                            len(sentence), len(tags), idx + 1))
                except Exception as e:
                    raise ValueError("exception raised when parsing line {}\n\t{}\n\t{}".format(idx + 1, line, e))

        xs, ys = np.asarray(xs), np.asarray(ys)

        # save train set
        cache_file = self.__cache_file_path(corpus_dir, max_seq_len)
        np.savez(cache_file, xs=xs, ys=ys)
        print("dataset cache({}, {}) => {}".format(xs.shape, ys.shape, cache_file))
        return xs, ys

## Définition du modèle et de la méthode d'entraînement 

D'abord la définition de la méthode qui instancie l'architecture. Les fichiers associés (_model_ et _arguments_) seront sauvés (ou chargés si un précédent entraînement a déjà eu lieu)  depuis _model_dir_.

Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/app/utils.py

In [ ]:
from os.path import exists, join
import torch

#FILE_ARGUMENTS = "arguments.json"
#FILE_MODEL = "model.pth"

def arguments_filepath(model_dir):
    return join(model_dir, FILE_ARGUMENTS)


def model_filepath(model_dir):
    return join(model_dir, FILE_MODEL)


def build_model(args, processor, load=True, verbose=False):
    model = BiRnnCrf(len(processor.vocab), len(processor.tags),
                     embedding_dim=args['embedding_dim'], hidden_dim=args['hidden_dim'], num_rnn_layers=args['num_rnn_layers'])

    # weights
    model_path = model_filepath(args['model_dir'])
    if exists(model_path) and load:
        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict)
        if verbose:
            print("load model weights from {}".format(model_path))
    return model


def running_device(device):
    if torch.cuda.is_available():
      print ('running_device gpu')
    else:  print ('running_device cpu')
    return device if device else torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Puis la définition des méthodes dédiées à l'entraînement du modèle

Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/app/train.py

In [ ]:
from os import mkdir
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

def __eval_model(model, device, dataloader, desc):
    model.eval()
    with torch.no_grad():
        # eval
        losses, nums = zip(*[
            (model.loss(xb.to(device), yb.to(device)), len(xb))
            for xb, yb in tqdm(dataloader, desc=desc)])
        return np.sum(np.multiply(losses, nums)) / np.sum(nums)


def __save_loss(losses, file_path):
    pd.DataFrame(data=losses, columns=["epoch", "batch", "train_loss", "val_loss"]).to_csv(file_path, index=False)


def __save_model(model_dir, model):
    model_path = model_filepath(model_dir)
    torch.save(model.state_dict(), model_path)
    print("save model => {}".format(model_path))


def train(args):
    model_dir = args['model_dir']
    if not exists(model_dir):
        mkdir(model_dir)
#    save_json_file(vars(args), arguments_filepath(model_dir))
    save_json_file(args, arguments_filepath(model_dir))

    preprocessor = Preprocessor(config_dir=args['corpus_dir'], save_config_dir=args['model_dir'], verbose=True)
    model = build_model(args, preprocessor, load=args['recovery'], verbose=True)

    # loss
    loss_path = join(args['model_dir'], "loss.csv")
    losses = pd.read_csv(loss_path).values.tolist() if args['recovery'] and exists(loss_path) else []

    # datasets
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = preprocessor.load_dataset(
        args['corpus_dir'], args['val_split'], args['test_split'], max_seq_len=args['max_seq_len'])
    train_dl = DataLoader(TensorDataset(x_train, y_train), batch_size=args['batch_size'], shuffle=True)
    valid_dl = DataLoader(TensorDataset(x_val, y_val), batch_size=args['batch_size'] * 2)
    test_dl = DataLoader(TensorDataset(x_test, y_test), batch_size=args['batch_size'] * 2)

    optimizer = optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    device = running_device(args['device'])
    model.to(device)

    val_loss = 0
    best_val_loss = 1e4
    for epoch in range(args['num_epoch']):
        # train
        model.train()
        bar = tqdm(train_dl)
        for bi, (xb, yb) in enumerate(bar):
            model.zero_grad()

            loss = model.loss(xb.to(device), yb.to(device))
            loss.backward()
            optimizer.step()
            bar.set_description("{:2d}/{} loss: {:5.2f}, val_loss: {:5.2f}".format(
                epoch+1, args['num_epoch'], loss, val_loss))
            losses.append([epoch, bi, loss.item(), np.nan])

        # evaluation
        val_loss = __eval_model(model, device, dataloader=valid_dl, desc="eval").item()
        # save losses
        losses[-1][-1] = val_loss
        __save_loss(losses, loss_path)

        # save model
        if not args['save_best_val_model'] or val_loss < best_val_loss:
            best_val_loss = val_loss
            __save_model(args['model_dir'], model)
            print("save model(epoch: {}) => {}".format(epoch, loss_path))

    # test
    test_loss = __eval_model(model, device, dataloader=test_dl, desc="test").item()
    last_loss = losses[-1][:]
    last_loss[-1] = test_loss
    losses.append(last_loss)
    __save_loss(losses, loss_path)
    print("training completed. test loss: {:.2f}".format(test_loss))



## Définition de la méthode de prédiction (utilisation du modèle)

La classe WordsTagger effectue l'étiquetage à proprement parler d'une nouvelle séquence de mots. Elle requiert le chemin vers un modèle.

Source : https://github.com/jidasheng/bi-lstm-crf/blob/master/bi_lstm_crf/app/predict.py

In [ ]:
import numpy as np

class WordsTagger:
    def __init__(self, model_dir, device=None):
        args = load_json_file(arguments_filepath(model_dir))
        #args = dict()
        args['model_dir'] = model_dir
        self.args = args

        self.preprocessor = Preprocessor(config_dir=model_dir, verbose=False)
        self.model = build_model(self.args, self.preprocessor, load=True, verbose=False)
        self.device = running_device(device)
        self.model.to(self.device)

        self.model.eval()

    def __call__(self, sentences, begin_tags="BS"):
        """predict texts
        :param sentences: a text or a list of text
        :param begin_tags: begin tags for the beginning of a span
        :return:
        """
        if not isinstance(sentences, (list, tuple)):
            raise ValueError("sentences must be a list of sentence")

        try:
            sent_tensor = np.asarray([self.preprocessor.sent_to_vector(s) for s in sentences])
            sent_tensor = torch.from_numpy(sent_tensor).to(self.device)
            with torch.no_grad():
                _, tags = self.model(sent_tensor)
            tags = self.preprocessor.decode_tags(tags)
        except RuntimeError as e:
            print("*** runtime error: {}".format(e))
            raise e
        return tags, self.tokens_from_tags(sentences, tags, begin_tags=begin_tags)

    @staticmethod
    def tokens_from_tags(sentences, tags_list, begin_tags):
        """extract entities from tags
        :param sentences: a list of sentence
        :param tags_list: a list of tags
        :param begin_tags:
        :return:
        """
        if not tags_list:
            return []

        def _tokens(sentence, ts):
            # begins: [(idx, label), ...]
            all_begin_tags = begin_tags + "O"
            begins = [(idx, t[2:]) for idx, t in enumerate(ts) if t[0] in all_begin_tags]
            begins = [
                         (idx, label)
                         for idx, label in begins
                         if ts[idx] != "O" or (idx > 0 and ts[idx - 1] != "O")
                     ] + [(len(ts), "")]

            tokens_ = [(sentence[s:e], label) for (s, label), (e, _) in zip(begins[:-1], begins[1:]) if label]
            return [((t, tag) if tag else t) for t, tag in tokens_]

        tokens_list = [_tokens(sentence, ts) for sentence, ts in zip(sentences, tags_list)]
        return tokens_list



---
## Entrainement effectif du modèle 

### Préparation des données d'entraînement WikiNER et des fichiers de configuration requis

Récupération des données d'entraînement Wikiner et prépation des fichiers de configuration : vocabulaire, étiquettes et données au format du code utilisé.

Après exécution de la cellule, consulter le répertoire `data` pour y trouver les fichiers tagset, vocab et txt produits pour le système NER précédemment défini.

In [ ]:
!mkdir -p data 
!wget -nc https://github.com/nicolashernandez/teaching_nlp/raw/main/data/wikiner_ud.joblib.bz2 -P data
!bzip2 -dk data/wikiner_ud.joblib.bz2

# Loading the corpus 
from joblib import load
wikiner_corpus = load('data/wikiner_ud.joblib') 

# Aperçu du nombre de phrases et d'une phrase annotée (liste de tokens composés de la forme, de la catégorie grammaticale et de l'étiquette BIO correspondant en l'entité nommée.
print (len(wikiner_corpus))
# 132257
print (wikiner_corpus[0]) 
vocab = set()
tagset = set()
for s in wikiner_corpus:
  for w,p,n in s:
    vocab.add(w.lower())
    tagset.add(n)
print (tagset)
# {'B-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'I-LOC', 'B-PER', 'I-PER', 'O'}

# export
with open('data/wikiner_corpus.txt', 'w', encoding='utf-8') as f:
    for line in wikiner_corpus:
      sentence = list()
      tags = list()    
      for w,p,n in line:
        sentence.append(w)
        tags.append(n)
      f.write('{}\t{}\n'.format(json.dumps(sentence), json.dumps(tags)))

# export tagset au format bi_lstm_crf 
import json
with open('data/wikiner_corpus_tagset.json', 'w', encoding='utf-8') as f:
    json.dump(list(tagset), f, ensure_ascii=False)

# export vocab au format bi_lstm_crf 
with open('data/wikiner_corpus_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(list(vocab), f, ensure_ascii=False)


Déclaration du répertoire de données et des noms des fichiers de vocab, du jeu d'étiquettes et du corpus étiquetés. En fait les noms des repertoires des données et du modèles sont définis un peu plus bas...

In [ ]:
FILE_VOCAB = "wikiner_corpus_vocab.json"
FILE_TAGS = "wikiner_corpus_tagset.json"
FILE_DATASET = "wikiner_corpus.txt"

Déclaration du répertoire du modèle qui sera généré 

In [ ]:
FILE_DATASET_CACHE = "dataset_cache_{}.npz"
FILE_ARGUMENTS = "arguments.json"
FILE_MODEL = "model.pth"

### Exécution de l'entraînement à partir de paramètres du modèle donnés



#### VOTRE TRAVAIL

* Avec le type d'exécution "cpu", l'entraînement peut prendre quelques minutes. Regardez le temps approximatif annoncé pour 1 époque. Passez en type "gpu" et comparez le temps.
* Avec les paramètres par défaut, quelle score de loss obtenez-vous pour les données de validation suite à la dernière époque d'entraînement ? Et sur les données de test ?

!!! Attention, l'implémentation cherchera à charger une configuration existante dans le répertoire du modèle spécifié. Si vous changez le paramétrage alors supprimer les fichiers spécifiques au modèle ou bien spécifier un nouveau répertoire pour le nouveau modèle.

In [ ]:
args = dict()
args['corpus_dir'] = "data"  # the corpus directory
args['model_dir'] = "model_wikiner_vanilla2"       # the output directory for model files
args['num_epoch'] = 5 # 5                # number of epoch to train
args['lr'] = 1e-3                     # learning rate
args['weight_decay'] = 0.             # the L2 normalization parameter
args['batch_size'] = 1000             # batch size for training
args['device'] = None                 # the training device: "cuda:0", "cpu:0". It will be auto-detected by default
args['max_seq_len'] = 100 # 100              # max sequence length within training
args['val_split'] = 0.2                  # the split for the validation dataset
args['test_split'] = 0.2                 # the split for the testing dataset
args['recovery'] = "store_true"       # continue to train from the saved model in model_dir
args['save_best_val_model'] = "store_true" # save the model whose validation score is smallest
args['embedding_dim'] = 100 # 100           # the dimension of the embedding layer
args['hidden_dim'] = 128              # the dimension of the RNN hidden state
args['num_rnn_layers'] = 21 # 1            # the number of RNN layers
args['rnn_type'] ="lstm"              # RNN type, choice: "lstm", "gru"
# print(args)

#
import time
start_time = time.time()

train(args)

print("--- %s seconds ---" % (time.time() - start_time))
# --- 162.1955807209015 seconds --- gpu 5 epochs max_seq_len 100 embedding_dim 100 num_rnn_layers 1 val_loss:  4.47 test_loss: 4.27

## Prédiction effective du modèle

### Préparation des données de tests WiNER

D'abord la définition de quelques méthodes utiles pour la préparation des données de tests

In [ ]:
# utilities 
def flatten(t):
  # applatie une liste de listes en une unique liste... 
  # [[a, b], [c], [d, e, f]] -> [a, b, c, d, e, f]
  return [item for sublist in t for item in sublist]

import re 
def normalise_labels(sentences):
  # normalise les sorties des étiquettes NER utilisées par les différents 
  # systèmes afin de les rendre comparable
  new_sentences = list()
  for sentence in sentences:
    new_sentence = list()
    for label in sentence:
      if label != 'O':
        label = re.sub('^[A-Z]-','', label)
      new_sentence.append(label)
    new_sentences.append(new_sentence)
  return new_sentences


Préparation des données

In [ ]:
# load the test corpus
!mkdir -p data
!wget -nc https://github.com/nicolashernandez/teaching_nlp/raw/main/data/winer_dev.joblib -P data
from joblib import load
winer_corpus = load('data/winer_dev.joblib')

# get the tokens of each text
# liste chaque forme de surface de chaque mot de chaque phrase
winer_tokens = [[token for token, pos, label in text] for text in winer_corpus]
# liste chaque étiquette (label) de chaque mot de chaque phrase
winer_ref = [[label for token, pos, label in text] for text in winer_corpus]
labels = list(set(flatten(winer_ref)))

#
print ('#texts:', len(winer_corpus))
print ('labels:', labels)

print ('sample of annotated texts:', winer_corpus[0])   
print ('sample of tokenized text:', winer_tokens[0])   

### Définition de la méthode d'évaluation

In [ ]:
# Measures definition
from sklearn.metrics import classification_report

def results_per_class(labels, y_ref, y_hyp):
  # Inspect per-class results in more detail:
  sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
  )
  # print ('y_ref', len(y_ref), 'y_hyp', len(y_hyp), 'sorted_labels', len(sorted_labels))
  return classification_report(flatten(y_ref), flatten(y_hyp), labels=sorted_labels, digits=3)

# Il y a beaucoup plus d'entités 'O' que les autres dans le corpus, 
# mais nous sommes davantage intéressés par les autres entités. 
# Pour ne pas biaiser les scores de moyenne, on retire les étiquettes qui ne nous intéressent pas.
print ("before removing:", labels)
labels_to_remove = ['O', 'Event', 'Date', 'Hour']
for l in labels_to_remove:
  if l in labels: labels.remove(l)
print ("after removing:", labels)

Prédiction sur une phrase exemple

In [ ]:
model = WordsTagger(model_dir="model_wikiner_vanilla")
tags, sequences = model([['George', 'W.', 'Bush', 'fut', 'président', 'des', 'États-Unis', "d'", 'Amérique', '.']])  # CHAR-based model
print(tags)  

### Exécution de la prédiction sur les données de test

In [ ]:
# predict
#from bi_lstm_crf.app import WordsTagger

import time
start_time = time.time()

bilstmcrf_model = WordsTagger(model_dir="model_wikiner_vanilla") #_vanilla

bilstmcrf_hyp = []
# pour chaque phrase de wikiner
for text in winer_tokens:
    tags, sequences = bilstmcrf_model([text])    
    bilstmcrf_hyp.append(tags[0])
    #print (tags)
    #break

#
print("--- %s seconds ---" % (time.time() - start_time))
# --- 40.24239158630371 seconds ---
# --- 144.3440752029419 seconds ---
# --- 33.92570495605469 seconds ---

# normalize the hyp labels
print (bilstmcrf_hyp[0])
normalized_bilstmcrf_hyp = normalise_labels(bilstmcrf_hyp)
print (normalized_bilstmcrf_hyp[0])
print (winer_ref[0])

# Evaluate on data 
print (results_per_class(labels, winer_ref, normalized_bilstmcrf_hyp))

avec les valeurs par défaut

```
running_device gpu
--- 49.18917155265808 seconds ---
['O', 'O', 'O', 'O', 'I-PER', 'I-MISC', 'O', 'O', 'O', 'O', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'O', 'O', 'O', 'I-PER', 'I-PER', 'O', 'O', 'I-LOC', 'O', 'O', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'I-LOC', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'I-LOC', 'O', 'O', 'O', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'PER', 'MISC', 'O', 'O', 'O', 'O', 'MISC', 'MISC', 'O', 'O', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'MISC', 'MISC', 'MISC', 'O', 'MISC', 'MISC', 'MISC', 'O', 'O', 'O', 'O', 'PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'LOC', 'O', 'O', 'MISC', 'MISC', 'MISC', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'O', 'O', 'LOC', 'O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'O', 'LOC', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'ORG', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'PER', 'PER', 'Date', 'Date', 'Date', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'ORG', 'O', 'O', 'O', 'O', 'Date', 'O', 'O', 'O', 'O', 'O', 'Date', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'ORG', 'O', 'O', 'PER', 'PER', 'PER', 'O', 'O', 'ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Date', 'O', 'O', 'O', 'ORG', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'O', 'O', 'O', 'O', 'ORG', 'O', 'O', 'O', 'O', 'Date', 'O']
              precision    recall  f1-score   support

         PER      0.287     0.659     0.400      4483
        MISC      0.012     0.625     0.024       443
         LOC      0.341     0.502     0.406      4724
         ORG      0.331     0.232     0.273      3816

   micro avg      0.154     0.482     0.233     13466
   macro avg      0.243     0.504     0.276     13466
weighted avg      0.309     0.482     0.354     13466
```



---
# VOTRE TRAVAIL

* Jouez avec le paramétrage d'entraînement du modèle (par exemple en doublant les valeurs par défaut): nombre d'époque, taille des phrases considérées (max_seq_len), dimension des embeddings (embedding_dim), nombre de couches RNN (num_rnn_layers), type de cellule RNN (rnn_type), nombre de dimension du RNN (hidden state), le taux d'apprentissage (lr)... Déterminer l'apport de chaque paramètre. Discuter les performances en termes de précision, rappel et micro/macro-F1.
* Dans vos expériences, rencontrez-vous des limites avec le hardware mis à disposition par gcolab ? 
* Expérimenter une modification en profondeur (au choix, d'autres sont possibles)
  * Modifiez le code pour utiliser des _pre-trained word embeddings_. Mesurez leur apport. Expérimentez à minima le modèle [word2vec de 200 dimensions construit avec skipgram sur le corpus FrWac, et mis à disposition par Jean-Philippe Fauconnier](https://fauconnier.github.io/#data)  
  * Ajouter les traits sur la surface des mots
* Faire un retour sur les différents modèles que vous avez implémentés (y compris à base de CRF pur).
* Suivant votre avancement, d'autres word embeddings peuvent être testés (e.g. glove), une architecture bi LSTM CNN CRF, voire un BERT-CRF...

Ci dessous quelques pointeurs sur comment utiliser des modèles pré-entraînés avec pytorch
* https://stackoverflow.com/questions/49710537/pytorch-gensim-how-to-load-pre-trained-word-embeddings
* https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76 
* https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f
